In [1]:
import os, glob, scipy
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import cv2
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout #Dense 층을 만들기위함(많이쓸듯)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint #좋은 모델 기억
from tensorflow.keras.models import load_model# 모델 불러오기
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
 
 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [2]:
model = load_model('model(M자).h5')

last_weight = model.layers[-1].get_weights()[0] 
   
new_model = Model(
    inputs=model.input,
    outputs=(
        model.layers[-3].output, 
        model.layers[-1].output
    )
)

new_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
#성공
from flask import Flask, redirect, render_template,request
from werkzeug.utils import secure_filename
import math

app =Flask(__name__)

@app.route('/picupload',methods = ['GET','POST'])
def upload_file():
    if request.method =='POST':
        file = request.files['file']
        #file.save(f'static/{secure_filename(file.filename)}')
        #img_path = f'static/{secure_filename(file.filename)}'
        fname = 'C:/Users/dz3kz/project(flask)/img\\'+str(secure_filename(file.filename))
        print(fname)
        file.save(fname)#이미지저장
        
       
        test_img = img_to_array(load_img(os.path.join(fname), target_size=(224, 224)))
        test_input = preprocess_input(np.expand_dims(test_img.copy(), axis=0))
        pred = model.predict(test_input)

        last_conv_output, pred = new_model.predict(test_input)

        last_conv_output = np.squeeze(last_conv_output) 
        feature_activation_maps = scipy.ndimage.zoom(last_conv_output, (32, 32, 1), order=1) 
    

        pred_class = np.argmax(pred) 
        predicted_class_weights = last_weight[:, pred_class] 

        final_output = np.dot(feature_activation_maps.reshape((224*224,1280)), predicted_class_weights).reshape(224,224)
    
        full=pred[0][0]
        middle = pred[0][1]
        free = pred[0][2]

        if full > middle and full > free:
            result=math.ceil(full)-1;
            result2='%.2f'%(full*100);
        elif middle > free: 
            result=math.ceil(middle);
            result2='%.2f'%(middle*100);
        else:
            result=math.ceil(free)+1;
            result2='%.2f'%(free*100);
            
        
        return redirect("http://localhost:8099/3Project/check.jsp?&result="+str(result)+"&result2="+str(result2))
    else:
        return render_template('picupload.jsp')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001, app)
    


 * Running on http://localhost:9001/ (Press CTRL+C to quit)


C:/Users/dz3kz/project(flask)/img\4.jpg


127.0.0.1 - - [26/Nov/2020 07:57:27] "POST /picupload HTTP/1.1" 302 -


C:/Users/dz3kz/project(flask)/img\1.jpg


127.0.0.1 - - [26/Nov/2020 07:57:43] "POST /picupload HTTP/1.1" 302 -


...

127.0.0.1 - - [26/Nov/2020 12:59:48] "POST /picupload HTTP/1.1" 302 -


C:/Users/dz3kz/project(flask)/img\26.jpg


127.0.0.1 - - [26/Nov/2020 13:08:16] "POST /picupload HTTP/1.1" 302 -


C:/Users/dz3kz/project(flask)/img\27.jpg


127.0.0.1 - - [26/Nov/2020 13:11:04] "POST /picupload HTTP/1.1" 302 -
